In [ ]:
from pycoingecko import CoinGeckoAPI
import pandas as pd
from datetime import datetime
import os
import requests
import json
from data_upload_utils import upload_to_github, update_airtable, create_airtable_record, delete_file_from_github

# Initialize CoinGecko
cg = CoinGeckoAPI()

# Validate environment variables
AIRTABLE_API_KEY = os.getenv("AIRTABLE_API_KEY")
GITHUB_TOKEN = os.getenv("GITHUB_TOKEN")
if not all([AIRTABLE_API_KEY, GITHUB_TOKEN]):
    print("❌ Missing environment variables: AIRTABLE_API_KEY or GITHUB_TOKEN")
    raise ValueError("Required environment variables are not set")

# Define coins
coins = {
    "BTC": "bitcoin",
    "ETH": "ethereum",
    "ADA": "cardano",
    "SOL": "solana",
    "DOT": "polkadot",
    "AVAX": "avalanche-2",
}

# Airtable configuration
BASE_ID = "appnssPRD9yeYJJe5"
TABLE_NAME = "Database"
AIRTABLE_URL = f"https://api.airtable.com/v0/{BASE_ID}/{TABLE_NAME}"
AIRTABLE_HEADERS = {
    "Authorization": f"Bearer {AIRTABLE_API_KEY}",
    "Content-Type": "application/json",
}

# GitHub configuration
GITHUB_REPO = "SagarFieldElevate/DatabaseManagement"
BRANCH = "main"
UPLOAD_PATH = "Uploads"

# Fetch 365 days of data and compute metrics
data = []
for symbol, coin_id in coins.items():
    try:
        market_data = cg.get_coin_market_chart_by_id(id=coin_id, vs_currency="usd", days=365)
        prices = market_data.get("prices", [])
        if not prices:
            print(f"⚠️ No price data for {symbol}")
            continue

        for i in range(1, len(prices)):
            prev_day = prices[i - 1]
            current_day = prices[i]

            prev_timestamp, prev_price = prev_day
            current_timestamp, current_price = current_day

            if not (prev_price > 0 and current_price > 0):
                print(f"⚠️ Invalid price data for {symbol} at {current_timestamp}")
                continue

            high = max(prev_price, current_price)
            low = min(prev_price, current_price)
            volatility = ((high - low) / low) * 100 if low > 0 else 0
            trading_range = high - low

            data.append({
                "symbol": symbol,
                "timestamp": datetime.utcfromtimestamp(current_timestamp / 1000).isoformat(),
                "high_24h_usd": round(high, 2),
                "low_24h_usd": round(low, 2),
                "volatility_24h_%": round(volatility, 2),
                "trading_range_24h_usd": round(trading_range, 2),
            })
    except Exception as e:
        print(f"❌ Error fetching data for {symbol}: {str(e)}")
        continue

# Create DataFrame and Excel file
if not data:
    print("❌ No data to process")
    raise ValueError("No data fetched from CoinGecko")

df = pd.DataFrame(data)
timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
filename = f"historical_volatility_trading_range_365_days_{timestamp}.xlsx"
df.to_excel(filename, index=False)
print(f"✅ Created Excel file: {filename}")

# Check Airtable for existing record
try:
    response = requests.get(
        AIRTABLE_URL,
        headers=AIRTABLE_HEADERS,
        params={"filterByFormula": "{Name}='365-Day Volatility and Range'"}
    )
    response.raise_for_status()
    data_airtable = response.json()
except Exception as e:
    print(f"❌ Airtable fetch failed: {str(e)}")
    if os.path.exists(filename):
        os.remove(filename)
    raise

# Get existing record
existing_records = [
    rec for rec in data_airtable.get("records", [])
    if rec["fields"].get("Name") == "365-Day Volatility and Range"
]

# Merge unique data
if existing_records:
    record_id = existing_records[0]["id"]
    existing_data = existing_records[0]["fields"].get("Data", [])
    print(f"✅ Found existing Airtable record: {record_id}")
else:
    record_id = None
    existing_data = []
    print("ℹ️ No existing Airtable record found")

# Ensure existing data has consistent format
for entry in existing_data:
    if isinstance(entry.get("timestamp"), datetime):
        entry["timestamp"] = entry["timestamp"].isoformat()

# Filter unique new data
existing_set = {(d["symbol"], d["timestamp"]) for d in existing_data}
unique_new_data = [d for d in data if (d["symbol"], d["timestamp"]) not in existing_set]
print(f"✅ Found {len(unique_new_data)} unique new entries out of {len(data)}")

# Combine data
updated_data = existing_data + unique_new_data

# Upload to GitHub
try:
    github_response = upload_to_github(filename, GITHUB_REPO, BRANCH, UPLOAD_PATH, GITHUB_TOKEN)
    raw_url = github_response["content"]["download_url"]
    file_sha = github_response["content"]["sha"]
    print("✅ Uploaded file to GitHub")
except Exception as e:
    print(f"❌ GitHub upload failed: {str(e)}")
    if os.path.exists(filename):
        os.remove(filename)
    raise

# Update or create Airtable record
try:
    if record_id:
        # Update existing record
        patch_url = f"{AIRTABLE_URL}/{record_id}"
        patch_payload = {
            "fields": {
                "Name": "365-Day Volatility and Range",
                "Data": updated_data,
                "Database Attachment": [{"url": raw_url, "filename": filename}],
            }
        }
        response = requests.patch(patch_url, headers=AIRTABLE_HEADERS, json=patch_payload)
    else:
        # Create new record
        post_payload = {
            "records": [{
                "fields": {
                    "Name": "365-Day Volatility and Range",
                    "Data": updated_data,
                    "Database Attachment": [{"url": raw_url, "filename": filename}],
                }
            }]
        }
        response = requests.post(AIRTABLE_URL, headers=AIRTABLE_HEADERS, json=post_payload)

    if response.status_code == 200:
        print("✅ Successfully updated/created Airtable record")
    else:
        print(f"❌ Airtable update failed: {response.status_code} - {response.text}")
        raise Exception("Airtable update failed")
except Exception as e:
    print(f"❌ Airtable operation failed: {str(e)}")
    if os.path.exists(filename):
        os.remove(filename)
    raise

# Cleanup
try:
    delete_file_from_github(filename, GITHUB_REPO, BRANCH, UPLOAD_PATH, GITHUB_TOKEN, file_sha)
    os.remove(filename)
    print("✅ Cleaned up local and GitHub files")
except Exception as e:
    print(f"❌ Cleanup failed: {str(e)}")
    raise